# Piecewise-Linear Regression

### `SMC` approach

In [1]:
# import libraries
from scipy.special import softmax

## basic imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import seaborn as sns
import cvxpy as cp
import time
clear_output()
myseed = '2404' 
np.random.seed(int(myseed))

In [2]:
## problem dimensions

B1 = int(6) 
B2 = int(5) 
L = 100.0 # X search space dim

AUGMENTED = True
RESCALE = True
TITLE = 'insurance'
TARGET_NAME = 'charges'

p_norm = 1
RB = 1e-1
RG = 1e-1
REL_PARAM = 1e-12

### utils

In [3]:
def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

In [4]:
'''
utils (global)
'''    

def extended_omegas(omegas,augmented=False):
    if augmented:
        N,d = omegas.shape
        embedding = np.zeros((N,int(d+d*(d+1)/2)))
        for s,samp in enumerate(omegas):
            new_omega = list(samp)
            buf = np.outer(samp,samp)
            for ofd in range(d):
                new_omega += list(np.diagonal(buf,ofd))
            embedding[s] = np.array(new_omega)
    else:
        embedding = omegas.copy()
    return np.hstack((embedding,np.ones((len(omegas),1))))

def rescale(data):
    if len(data.shape)==2:
        num_samples,num_features = data.shape
        new_data = data.copy()
        means,stds = [],[]
        for colid,feat in enumerate(data.T):
            mfeat,stdfeat = np.mean(feat),np.std(feat)
            means.append(mfeat)
            stds.append(stdfeat)
            if stdfeat>0:
                new_data[:,colid] = (feat-mfeat)/stdfeat
            else:
                means[colid] = 0
                stds[colid] = 1
        return new_data,means,stds
    else:
        mean = np.mean(data)
        std = np.std(data)
        if std>0:
            return (data-mean)/std,mean,std
        else:
            return data,0,1

def coordinates(sigma_,B1_=B1,B2_=B2):
    assert sigma_>=0 and sigma_<=B1_*B2_-1, 'value range error (0 -> H-1)'
    e2 = sigma_%B2_
    e1 = (sigma_-(e2-1))/B2_
    return (int(e1),int(e2))

def selector(coordinates,B1_=B1,B2_=B2):
    return coordinates[1]+coordinates[0]*B2_

def recast(x,w_ext,B1_=B1,B2_=B2):
    assert len(x)==(B1_+B2_)*w_ext, 'dimension error'
    mat = x.reshape((B1_+B2_,w_ext))
    return mat[:B1_],mat[B1_:]

def mix(mat1,mat2):
    assert mat1.shape[1]==mat2.shape[1],'dimension error'
    buf = []
    for lm1 in mat1:
        for lm2 in mat2:
            buf.append(list(lm1+lm2))
    return np.array(buf)

def unfold(mat1,mat2):
    return (np.vstack((mat1,mat2))).flatten()

### true data generation

In [5]:
def preprocess(fname,tf,augmented=True):
    ## loading part 
    try:
        raw_df = pd.read_csv(fname)
        raw_df.dropna(inplace=True)
        if isinstance(tf,str):
            target_df = raw_df[tf]
        elif isinstance(tf,numbers.Number):
            target_df = raw_df.iloc[:,int(tf)]
        else:
            print('tf should either be a column name (str) or location (int)')
            return None,None
        target = target_df.values
        data_df = raw_df.drop(columns=[tf])
        obj_df = data_df.select_dtypes(include='object')
        to_categorize = list(obj_df.columns)
        for feat_str in to_categorize:
            if isinstance(obj_df[feat_str].values[0],str):
                cats = obj_df[feat_str].unique()
                ncats = len(cats)
                if ncats%2==0:
                    vals = np.linspace(-int(ncats/2),int(ncats/2),ncats)
                else:
                    vals = np.linspace(-int((ncats-1)/2),int((ncats)/2),ncats)
                vals /= max(abs(vals))
                dic_rep = {}
                for cat,num in zip(cats,vals):
                    dic_rep[cat] = num
                data_df[feat_str].replace(dic_rep,inplace=True)
        data = data_df.select_dtypes('number').values
    except:
        print('-> error while loading your file')
        return None,None
        
    data = extended_omegas(data,augmented) # adds "bias terms"
    
    return target,data

In [6]:
pre_taus,pre_omegas = preprocess(TITLE+'.csv',TARGET_NAME,AUGMENTED)
if RESCALE:
    taus,meantaus,stdtaus = rescale(pre_taus)
    omegas,meanomegas,stdomegas = rescale(pre_omegas)
else:
    taus = pre_taus.copy()
    omegas = omegas.copy()

In [7]:
N_train = min(len(omegas),int(750))
id_train = np.random.choice(np.arange(len(omegas)),replace=False,size=N_train)

omegas_train = omegas[id_train]
taus_train = taus[id_train]
w_ext = omegas_train.shape[1]

gamma_bound = np.max(np.abs(taus_train))

### TRAINING

`parametric LP` 

In [8]:
# symmetry breakers tolerances
margin = .0

# variables
mat1_cvx = cp.Variable((B1,w_ext))
mat2_cvx = cp.Variable((B2,w_ext))

# parameters
param1 = cp.Parameter((B1,w_ext))
param2 = cp.Parameter((B2,w_ext))

# basic feasible set 
X = [cp.norm(mat1_cvx[e1][:-1],'inf')<=L for e1 in range(B1)] + [cp.norm(mat2_cvx[e2][:-1],'inf')<=L for e2 in range(B2)] 

# prior-knowledge encoding
symmetry_breaks = []
if B1>1:
    for e1 in range(B1-1):
        symmetry_breaks += [cp.sum(mat1_cvx[e1])+margin<=cp.sum(mat1_cvx[e1+1])]
if B2>1:
    for e2 in range(B2-1):
        symmetry_breaks += [cp.sum(mat2_cvx[e2])+margin<=cp.sum(mat2_cvx[e2+1])]

# objective function implementation
val1_cvx,val2_cvx = mat1_cvx@omegas_train.T,mat2_cvx@omegas_train.T
mval1_cvx,mval2_cvx = cp.max(val1_cvx,0),cp.max(val2_cvx,0)
vec_h_bar_cvx = cp.maximum(taus_train+mval2_cvx,mval1_cvx)+cp.maximum(-taus_train+mval1_cvx,mval2_cvx)
h_bar_smc = 1/N_train * cp.sum(vec_h_bar_cvx)
F_param_smc = h_bar_smc-(1/N_train)*(cp.sum(cp.multiply(param1,mat1_cvx))\
                                               +cp.sum(cp.multiply(param2,mat2_cvx)))

prob_param_smc = cp.Problem(cp.Minimize(F_param_smc),X+symmetry_breaks)

In [9]:
'''-> ERM evaluation<-'''
def Feval(mat1_val,mat2_val,data=omegas_train,target=taus_train):
    val1,val2=mat1_val@data.T,mat2_val@data.T
    return np.mean(np.abs(target-(np.max(val1,0)-np.max(val2,0))))

'''-> h components evaluation <-'''
def heval(mat1_val,mat2_val,data=omegas_train):
    loc_B1,loc_B2 = len(mat1_val),len(mat2_val)
    val1,val2=mat1_val@data.T,mat2_val@data.T
    buf = np.zeros((len(data),loc_B1*loc_B2))
    for e1,v1 in enumerate(val1):
        buf[:,e1*loc_B2:(e1+1)*loc_B2] = -(np.outer(v1,np.ones(loc_B2))+val2.T)
    return buf

''' -> active components <- '''
def rho_active_sets(vals,rho=1e-10):
    indices = []
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    shifted_vals = (np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1])))
    for idrow,row in enumerate(shifted_vals):
        tokeep = len(np.where(row>=1-rho)[0])
        indices.append(list(np.argsort(vals[idrow]))[:tokeep])
    return indices

In [10]:
'''-> computation of parameters <-
weights ndarray of shape (N_train,B1*B2)
'''
def w2p(weights):
    param1_val = np.zeros((B1,w_ext))
    param2_val = np.zeros((B2,w_ext))
    for l in range(B1*B2):
        e1_sel,e2_sel = coordinates(l)
        vec_shift = weights[:,l]@omegas_train
        param1_val[e1_sel]+=vec_shift
        param2_val[e2_sel]+=vec_shift
    param1.value = param1_val
    param2.value = param2_val
    
    
''' weights computations '''

def Q_opt(vals):
    buf = np.zeros(vals.shape)
    for s,sigma_s in enumerate(np.argmin(vals,1)):
        buf[s,sigma_s] += 1
    return buf

def Q_safe(vals,Q_current,Q_hat,Q_star,C):
    Q_next = np.zeros(Q_star.shape)
    combinations = np.zeros(len(Q_current))
    for _,Q_hat_elem in enumerate(Q_hat):
        num = np.sum((Q_current[_]-Q_star[_])*vals[_])
        denum = np.sum((Q_hat_elem-Q_star[_])*vals[_])
        if C*num>=denum:
            combinations[_] = 1
        else:
            combinations[_] = min(1,max(0,C*(num/max(1e-9,denum))))
        Q_next[_] = combinations[_]*Q_hat_elem+(1-combinations[_])*Q_star[_]
    return Q_next,combinations

def Q_hat_bb(vals,Q,param=1):
    minids = np.argmin(vals,1)
    Q_hat = Q.copy()
    for s,sigma_s in enumerate(minids):
        Q_hat[s] -= param*np.ones(vals.shape[1])
        Q_hat[s,sigma_s] += 2*param
    return proj_simplex_vec(Q_hat)

def Q_hat_mm(vals,param=1):
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    return proj_simplex_vec(param*(np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1]))))

def Q_hat_softmin(vals,param=1,num_pert_inf=5e-7):
    pert_vals = vals+np.random.uniform(-num_pert_inf,num_pert_inf,vals.shape)
    return softmax(-param*pert_vals/np.maximum(1e-4,np.abs(np.outer(np.mean(vals,1),np.ones(B1*B2)))),axis=1)

For any $(l,l_+) \in [n_s]^2$,

$$M^{(s)}_{l_+,l} = \max_{u \,\in\, \mathcal{S}}\, h^{(s)}_{l_+}(u)-h^{(s)}_{l}(u) \quad \text{s.t.} \quad h^{(s)}_{l}(u)\leq h^{(s)}(u)$$


<b>PLR</b>

* $\mathcal{S} = \mathbb{B}_{||\cdot||}(r_\beta\,;\,c^{(1)}_{1})\times [\upsilon^{(1)}_1-r_\gamma,\upsilon^{(1)}_1+r_\gamma] \times \dots \times \mathbb{B}_{||\cdot||}(r_\beta\,;\,c^{(1)}_{B_1}) \times  [\upsilon^{(1)}_{B_1}-r_\gamma,\upsilon^{(1)}_{B_1}+r_\gamma] \times \mathbb{B}_{||\cdot||}(r_\beta\,;\,c^{(2)}_{1}) \times  [\upsilon^{(2)}_1-r_\gamma,\upsilon^{(2)}_1+r_\gamma]\times \dots \times \mathbb{B}_{||\cdot||}(r_\beta\,;\,c^{(2)}_{B_2})\times  [\upsilon^{(2)}_{B_2}-r_\gamma,\upsilon^{(2)}_{B_2}+r_\gamma]$ 


*  $h^{(s)}_{l}(u) = -\big[\langle \beta^{(1)}_{e_1(l)}+\beta^{(2)}_{e_2(l)}, w^{(s)}\rangle + \big(\gamma^{(1)}_{e_1(l)}+\gamma^{(2)}_{e_2(l)}\big)\big] \quad \text{with} \quad u^T = \big((\beta_{1}^{(1)})^T,\gamma_{1}^{(1)},\dots,(\beta_{B_1}^{(1)})^T,\gamma_{B_1}^{(1)},(\beta_{1}^{(2)})^T,\gamma_{1}^{(2)},\dots,(\beta_{B_2}^{(2)})^T,\gamma_{B_2}^{(2)}\big)$


* $ \hat{M}^{(s)}_{l_+,l} = \Big(\big(\upsilon^{(1)}_{e_1(l)}-\upsilon^{(1)}_{e_1(l_+)}\big)+\big(\upsilon^{(2)}_{e_2(l)}-\upsilon^{(2)}_{e_2(l_+)}\big)\Big) + \big\langle \omega^{(s)},\big(c^{(1)}_{e_1(l)}-c^{(1)}_{e_1(l_+)}\big)+\big(c^{(2)}_{e_2(l)}-c^{(2)}_{e_2(l_+)}\big)\big\rangle + \\2\cdot\Big(\mathbf{1}_{\{e_1(l_+)\not=e_1(l)\}}+\mathbf{1}_{\{e_2(l_+)\not=e_2(l)\}}\Big)\cdot\big[r_\beta\, ||\omega^{(s)}||_* + r_\gamma\big]$ 

<i>NOTE</i>: 

* if there exists $l_+ \in [n_s]$ such that $\hat{M}^{(s)}_{l_+,l}<0$ then $l$ is <b>not</b> active at any minimizer of $F$ over $\mathcal{S}$ and one sets $M^{(s)}_{l_+,l}=0$
* else, one sets $M^{(s)}_{l_+,l}=\hat{M}^{(s)}_{l_+,l}$

In [11]:
def bigM_PLR(OMEGAS,B1param=B1,B2param=B2,r_beta=L,r_gamma=gamma_bound,upsilon_centers=None,c_centers=None,p=2):
    
    print('-> computation of r_beta = '+str(r_beta)+' ; r_gamma = '+str(r_gamma)+' ; p-norm with p = '+str(p))
    print(' ')
    
    # init 
    N_loc = len(OMEGAS)
    data_loc = OMEGAS[:,:-1]
    M_list = [np.zeros((B1param*B2param,B1param*B2param)) for _ in range(N_loc)]
    if upsilon_centers is None:
        upsilon_centers = np.zeros((B1param+B2param))
    if c_centers is None:
        c_centers = np.zeros((B1param+B2param,data_loc.shape[1]))
        
    # dual q-norm with 1/p+1/q=1 <=> 1/(1-1/p) (p>=1)
    if p<=1:
        dual_norm = lambda omega: np.max(np.abs(omega))
    else:
        q = 1/(1-1/p)
        dual_norm =lambda omega: (np.sum(np.abs(omega)**q))**(1/q)
        
    # main loop
    for s,omega in enumerate(data_loc):
        for l_plus in range(B1param*B2param):
            e1_sel_plus,e2_sel_plus = coordinates(l_plus)
            for l in range(B1param*B2param):
                if l==l_plus:
                    pass
                else:
                    # computations
                    e1_sel,e2_sel = coordinates(l)
                    e1_equal,e2_equal = np.sum(e1_sel_plus!=e1_sel),np.sum(e2_sel_plus!=e2_sel)
                    diff_upsilon_1 = upsilon_centers[e1_sel]-upsilon_centers[e1_sel_plus]
                    diff_upsilon_2 = upsilon_centers[B1param+e2_sel]-upsilon_centers[B1param+e2_sel_plus]
                    diff_c_1 = c_centers[e1_sel]-c_centers[e1_sel_plus]
                    diff_c_2 = c_centers[B1param+e2_sel]-c_centers[B1param+e2_sel_plus]
                    M_list[s][l_plus,l] = 2*(e1_equal+e2_equal)*(r_beta*dual_norm(omega)+r_gamma) + (diff_upsilon_1+diff_upsilon_2)+np.sum(omega*(diff_c_1+diff_c_2))
        if (s+1)%(np.ceil(N_loc/10))==0:
            print('ok for '+str((s+1)/N_loc*100)+'%')
    return M_list

In [12]:
NREPS = int(5)
Q_init_mats = []
for rep in range(NREPS):
    Q_init = np.random.uniform(0,1,(N_train,B1*B2))
    Q_init = np.array([q/np.sum(q) for q in Q_init])
    Q_init_mats.append(Q_init)

N_outer_max = int(10)
N_max = int(400)
tol_loc,tol_glob = 1e-8,5e-7

filter_out = False

import gurobipy

local_env = gurobipy.Env()
local_env.setParam('TimeLimit', 120) # in seconds
local_env.setParam('Heuristics',.25) # in %

Academic license - for non-commercial use only - expires 2024-09-05
Using license file /Users/guiom_vds/gurobi.lic
Changed value of parameter TimeLimit to 120.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Heuristics to 0.25
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05


In [13]:
def RRAM(mode='am'):

    ## init
    fval = []
    t = []
    repIDs = []
    locopt = []
    endval_single_run = []
    endval = [] 

    if mode=='maxmin':
        param_schedule = lambda k: k**(2/3) 
        C_schedule = lambda k: 2/(k**(1/2)+3)
    elif mode=='softmin':
        param_schedule = lambda k: (3/2)**(k**(3/4)) if (3/2)**(k**(3/4))<1e16 else np.inf
        C_schedule = lambda k: 2/(k**(1/2)+3)
    elif mode=='baratt-boyd':
        param_schedule = lambda k: 1e-1
        C_schedule = lambda k: 1
    else: # am
        param_schedule = lambda k: 1
        C_schedule = lambda k: 0 

    print('=> '+str(mode)+' (START)')
    print(' ')

    for rep in range(NREPS):
        print('new rep #'+str(rep+1))
        print(' ')

        t_init = time.time()
        f_overallbest = np.inf
        mat1_out,mat2_out = None,None
        LO = False

        for k_outer in range(N_outer_max):

            print('OUTER #'+str(k_outer+1))
            print(' ')

            if k_outer==0:
                Q = Q_init_mats[rep].copy()
                w2p(Q)
                ref_val = np.inf
            else:
                components_ = heval(mat1_cvx.value,mat2_cvx.value)
                Q = Q_opt(components_)
                w2p(Q)
                ref_val = Feval(mat1_cvx.value,mat2_cvx.value)
                print('restart w/ F = '+str(Feval(mat1_cvx.value,mat2_cvx.value)))
                print(' ')

            ### LOCAL

            try:

                print('local')
                print(' ')

                ## main loop
                for k in range(N_max):

                    C,kappa = C_schedule(k),param_schedule(k)

                    #### set x 
                    prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

                    #### set weights
                    components_ = heval(mat1_cvx.value,mat2_cvx.value)
                    prev = np.sum(components_/N_train*Q)
                    if mode=='maxmin':
                        Q_hat = Q_hat_mm(components_,param=kappa)
                        Q_star = Q_opt(components_)
                        Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
                    elif mode=='softmin':
                        Q_hat = Q_hat_softmin(components_,param=kappa)
                        Q_star = Q_opt(components_)
                        Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
                    elif mode=='baratt-boyd':
                        Q,combis = Q_hat_bb(components_,Q,param=kappa),np.array([C])
                    else:
                        Q,combis = Q_opt(components_),np.array([C])
                    w2p(Q)
                    post = np.sum(components_/N_train*Q)
                    F_Q_plus_x_plus = F_param_smc.value
                    G_crit = max(0,prev-post)
                    F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                    print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. eps. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
                    if (ref_val-F_)<tol_loc and k>10:
                        print('=> convergence to criticality ... polishing phase')
                        Q_final = Q_opt(heval(mat1_cvx.value,mat2_cvx.value))
                        w2p(Q_final)
                        prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                        F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                        print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                        break;
                    else:
                        ref_val = F_Q_plus_x_plus
                if k==N_max-1:
                    F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            except: 
                print('... solver failed | early interruption ')
                F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            print(' ')
            if F_<=f_overallbest:
                print(' /!\ NEW BEST | F = '+str(F_)+'  /!\ ')
                f_overallbest = F_
                mat1_out = mat1_cvx.value
                mat2_out = mat2_cvx.value
                print(' ')
            if k_outer==0:
                endval_single_run.append(F_)
            t.append(time.time()-t_init)
            fval.append(F_)
            repIDs.append(rep)

            ### GLOBAL
            # @check in order not to cycle...
            active_indices = rho_active_sets(heval(mat1_out,mat2_out),rho=1e-12)
            val1,val2=mat1_out@omegas_train.T,mat2_out@omegas_train.T
            individual_losses = np.abs(taus_train-np.max(val1,0)+np.max(val2,0))
            print('global')
            print(' ')
            try:
                upsilons = np.concatenate((mat1_out[:,-1],mat2_out[:,-1]))
                cs = np.vstack((mat1_out[:,:-1],mat2_out[:,:-1]))
                list_cstr_glob_cvx = []
                list_cstr_glob_cvx += [cp.norm(mat1_cvx[e1][:-1]-mat1_out[e1][:-1],p_norm)<=RB for e1 in range(B1)]
                list_cstr_glob_cvx += [cp.norm(mat2_cvx[e2][:-1]-mat2_out[e2][:-1],p_norm)<=RB for e2 in range(B2)]
                list_cstr_glob_cvx += [cp.abs(mat1_cvx[e1][-1]-mat1_out[e1][-1])<=RG for e1 in range(B1)]
                list_cstr_glob_cvx += [cp.abs(mat2_cvx[e2][-1]-mat2_out[e2][-1])<=RG for e2 in range(B2)]
                ML_loc = bigM_PLR(OMEGAS=omegas_train,r_beta=RB,r_gamma=RG,upsilon_centers=upsilons,c_centers=cs,p=p_norm)

                fun_obj_glob_cvx = h_bar_smc
                t_var = {} 
                for s,omega_s in enumerate(omegas_train):
                    active_indices_s = active_indices[s]
                    n_a_s = len(active_indices_s)
                    l_elected = active_indices_s[0]
                    e1_elected,e2_elected = coordinates(l_elected)
                    if n_a_s>1 and (filter_out==False or individual_losses_maxmin[s]<1e-7):
                        t_var['t_'+str(s)] = cp.Variable(n_a_s,boolean=True)
                        alpha_s = cp.Variable(1,name='alpha_'+str(s))
                        list_cstr_glob_cvx += [cp.sum(t_var['t_'+str(s)])==1]
                        init_t = np.zeros(n_a_s)
                        init_t[0] += 1
                        t_var['t_'+str(s)].value = init_t
                        fun_obj_glob_cvx += alpha_s[0]
                        alpha_s.value = np.array([-1/N_train*omega_s@(mat1_cvx.value[e1_elected]+mat2_cvx.value[e2_elected])])
                        for l_plus in active_indices_s:
                            e1_sel,e2_sel = coordinates(l_plus)
                            M_bounds = ML_loc[s][l_plus]
                            list_cstr_glob_cvx += [-1/N_train * omega_s@(mat1_cvx[e1_sel]+mat2_cvx[e2_sel])<=alpha_s+1/N_train*cp.sum(cp.multiply(t_var['t_'+str(s)],M_bounds[active_indices_s]))]
                    else:
                        fun_obj_glob_cvx += -1/N_train * omega_s@(mat1_cvx[e1_elected]+mat2_cvx[e2_elected])

                prob_glob_cvx = cp.Problem(cp.Minimize(fun_obj_glob_cvx),list_cstr_glob_cvx+X+symmetry_breaks)
                local_env.setParam('BestObjStop',f_overallbest*(1-.02*np.sign(f_overallbest)))
                prob_glob_cvx.solve(solver=cp.GUROBI,verbose=True,warm_start=True,env=local_env)
                F_ =  Feval(mat1_cvx.value,mat2_cvx.value)
            except:
                print(' ')
                print('GUROBI failure or manual STOP...')
                F_ = np.inf

            if F_ <= f_overallbest-tol_glob:
                print(' ')
                print(' /!\ NEW BEST | F = '+str(F_)+'  /!\ ')
                print(' ')
                f_overallbest = F_
                t.append(time.time()-t_init)
                fval.append(F_)
                repIDs.append(rep)
            else:
                print(' ')
                print('no improvement after globalization technique... ')
                LO = prob_glob_cvx.status=='optimal'
                break;
        print(' ')
        print('=> '+str(mode)+' (END)')
        print(' ')
        locopt.append(LO)
        endval.append(f_overallbest)
        
    return t,fval,repIDs,locopt,endval,endval_single_run

In [14]:
t_sm,fval_sm,repIDs_sm,locopt_sm,endval_sm,endval_single_run_sm = RRAM(mode='softmin')

=> softmin (START)
 
new rep #1
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 3.2781e-13 | avg. eps. 2.1859e-04 
iter. 0002 | Fval. 1.6281e-01 | BICval. 1.6423e-01 | Gcrit_x. 1.4652e-03 | avg. eps. 2.7831e-02 
iter. 0003 | Fval. 1.4937e-01 | BICval. 1.5448e-01 | Gcrit_x. 6.4627e-03 | avg. eps. 3.7931e-02 
iter. 0004 | Fval. 1.3976e-01 | BICval. 1.4407e-01 | Gcrit_x. 5.9605e-03 | avg. eps. 2.8853e-02 
iter. 0005 | Fval. 1.3225e-01 | BICval. 1.3530e-01 | Gcrit_x. 4.9270e-03 | avg. eps. 3.5939e-02 
iter. 0006 | Fval. 1.2818e-01 | BICval. 1.3000e-01 | Gcrit_x. 3.7852e-03 | avg. eps. 2.5955e-02 
iter. 0007 | Fval. 1.2481e-01 | BICval. 1.2596e-01 | Gcrit_x. 1.9784e-03 | avg. eps. 1.2488e-02 
iter. 0008 | Fval. 1.2080e-01 | BICval. 1.2171e-01 | Gcrit_x. 1.7507e-03 | avg. eps. 3.2718e-02 
iter. 0009 | Fval. 1.1971e-01 | BICval. 1.2018e-01 | Gcrit_x. 9.1137e-04 | avg. eps. 6.0076e-03 
iter. 0010 | Fval. 1.1782e-01 | BICval. 1.1822e-01 | Gcrit_x. 8.0816e-04 |

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20925 rows, 5300 columns and 494735 nonzeros
Model fingerprint: 0x04ab2e27
Variable types: 5182 continuous, 118 integer (118 binary)
Coefficient statistics:
  Matrix range     [2e-06, 5e+00]
  Objective range  [7e-06, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-03, 3e+02]

User MIP start produced solution with objective 0.0749721 (0.13s)
Loaded user MIP start with objective 0.0749721

Presolve removed 11992 rows and 3854 columns
Presolve time: 2.00s
Presolved: 8933 rows, 1446 columns, 239937 nonzeros
Variable types: 1378 continuous, 68 integer (68 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4122   -4.0390011e+02  

Coefficient statistics:
  Matrix range     [3e-05, 5e+00]
  Objective range  [6e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-03, 3e+02]

User MIP start produced solution with objective 0.0736215 (0.12s)
Loaded user MIP start with objective 0.0736215

Presolve removed 11950 rows and 3812 columns
Presolve time: 1.84s
Presolved: 8823 rows, 1336 columns, 234100 nonzeros
Found heuristic solution: objective 0.0736215
Variable types: 1329 continuous, 7 integer (7 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4339   -7.4830370e+01   4.612260e+05   0.000000e+00      5s
   12578    6.6745693e-02   2.163278e+00   0.000000e+00     10s
   17298    7.0588931e-02   0.000000e+00   0.000000e+00     13s

Root relaxation: objective 7.058893e-02, 17298 iterations, 10.67 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

(CVXPY) Jul 24 02:52:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 24 02:52:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 24 02:52:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 02:52:14 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Jul 24 02:52:14 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 02:52:14 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 02:52:14 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 02:52:14 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 02:52:15 PM: Applying reduction GUROB

(CVXPY) Jul 24 02:55:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 02:55:06 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Jul 24 02:55:06 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 02:55:06 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 02:55:06 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 02:55:06 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 02:55:08 PM: Applying reduction GUROBI
(CVXPY) Jul 24 02:55:08 PM: Finished problem compilation (took 2.585e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver  

...

H    0     0                       0.0802698    0.06914  13.9%     -   15s
     0     0    0.06914    0   29    0.08027    0.06914  13.9%     -   16s
H    0     0                       0.0802698    0.06914  13.9%     -   17s
     0     2    0.06914    0   29    0.08027    0.06914  13.9%     -   19s
     1     4    0.06914    1   29    0.08027    0.06914  13.9%   2.0   20s
    31    38    0.07197    7   23    0.08027    0.06952  13.4%   480   27s
    37    44    0.07258    8   21    0.08027    0.06952  13.4%   542   30s
H   38    44                       0.0802695    0.06952  13.4%   527   30s
    59    67    0.07413   11   16    0.08027    0.06952  13.4%   597   35s
    86   100    0.07516   13   14    0.08027    0.06952  13.4%   567   40s
   181   211    0.07907   23    4    0.08027    0.06952  13.4%   369   46s
*  216   234              25       0.0802694    0.06952  13.4%   332   46s
*  242   255              25       0.0802694    0.06952  13.4%   309   48s
*  244   255             

     0     2    0.06449    0   40    0.08001    0.06449  19.4%     -   24s
     3     8    0.06497    2   40    0.08001    0.06449  19.4%  92.7   25s
    31    38    0.06860    7   32    0.08001    0.06544  18.2%   418   34s
    37    44    0.06936    8   30    0.08001    0.06544  18.2%   399   38s
    43    52    0.07007    9   28    0.08001    0.06544  18.2%   421   40s
    97   116    0.07231   14   22    0.08001    0.06544  18.2%   355   45s
   145   167    0.07494   18   14    0.08001    0.06544  18.2%   323   50s
   262   270    0.07945   30    2    0.08001    0.06544  18.2%   228   55s
   326   326    0.06784    7   34    0.08001    0.06544  18.2%   218   61s
   366   345    0.07007   10   28    0.08001    0.06544  18.2%   220   65s
   397   383    0.07344   15   19    0.08001    0.06544  18.2%   236   71s
   439   409    0.07816   26    6    0.08001    0.06544  18.2%   242   75s
H  472   415                       0.0800101    0.06552  18.1%   239   77s
H  491   415             

*  477   281              19       0.0798009    0.07344  7.97%   111   47s
   594   327    0.07924   17    2    0.07980    0.07344  7.97%   101   50s
H  712   335                       0.0798009    0.07344  7.97%  90.7   53s
   747   386    0.07924   17    2    0.07980    0.07344  7.97%  93.1   56s
   900   418    0.07861   15    4    0.07980    0.07344  7.97%  88.9   60s
   937   439    0.07954   18    1    0.07980    0.07344  7.97%  90.7   69s
   982   436    0.07954   18    1    0.07980    0.07378  7.55%  91.2   71s
  1072   511    0.07630   10   10    0.07980    0.07460  6.52%  93.1   75s
  1267   596    0.07587    9   11    0.07980    0.07463  6.48%  91.7   80s
  1541   714    0.07587   10   11    0.07980    0.07483  6.23%  89.5   85s
  1682   747    0.07954   20    1    0.07980    0.07496  6.06%  89.3  106s
  1836   814    0.07630   11   10    0.07980    0.07496  6.06%  89.0  110s
  2072   854    0.07587   10   11    0.07980    0.07503  5.98%  86.9  115s
  2486   854    0.07954  

   584   468    0.07699   22    8    0.07944    0.06764  14.8%   213   71s
   616   519    0.07765   24    6    0.07944    0.06764  14.8%   220   75s
   687   557    0.07861   26    3    0.07944    0.06764  14.8%   219   82s
   722   566    0.07797   25    5    0.07944    0.06764  14.8%   220   86s
   797   655    0.07861   27    3    0.07944    0.06764  14.8%   216   90s
   922   743    0.07917   29    1    0.07944    0.06764  14.8%   205   95s
   994   758    0.07917   29    1    0.07944    0.06764  14.8%   207  104s
  1007   784     cutoff   30         0.07944    0.06764  14.8%   208  107s
H 1047   804                       0.0794354    0.06764  14.8%   210  110s
* 1120   878              37       0.0794353    0.06764  14.8%   207  113s
  1156   899    0.07227   17   21    0.07944    0.06764  14.8%   205  115s
  1238   948    0.06907    8   29    0.07944    0.06803  14.4%   213  120s

Explored 1279 nodes (290030 simplex iterations) in 120.04 seconds
Thread count was 12 (of 12 availa


  Push phase complete: Pinf 5.3787859e-06, Dinf 6.3431216e-10      1s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     942    7.8977148e-02   5.378786e-06   0.000000e+00      2s
     947    7.8977147e-02   0.000000e+00   0.000000e+00      2s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 2s

     199 PPushes remaining with PInf 0.0000000e+00                 2s
       0 PPushes remaining with PInf 0.0000000e+00                 2s

  Push phase complete: Pinf 0.0000000e+00, Dinf 1.4958079e-10      2s

Extra simplex iterations from dual to original model: 110
Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1594    7.8977147e-02   0.000000e+00   0.000000e+00      2s

Solved with barrier
Solved in 1594 iterations and 1.94 seconds
Optimal objective  7.897714747e-02
-------------------------------------------------------------------------------
 

In [15]:
t_am,fval_am,repIDs_am,locopt_am,endval_am,endval_single_run_am = RRAM(mode='am')

=> am (START)
 
new rep #1
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 3.2790e-13 | avg. eps. 0.0000e+00 
iter. 0002 | Fval. 1.6277e-01 | BICval. 1.6277e-01 | Gcrit_x. 2.9011e-03 | avg. eps. 0.0000e+00 
iter. 0003 | Fval. 1.4945e-01 | BICval. 1.4945e-01 | Gcrit_x. 8.1030e-03 | avg. eps. 0.0000e+00 
iter. 0004 | Fval. 1.3756e-01 | BICval. 1.3756e-01 | Gcrit_x. 4.8910e-03 | avg. eps. 0.0000e+00 
iter. 0005 | Fval. 1.3307e-01 | BICval. 1.3307e-01 | Gcrit_x. 1.4731e-03 | avg. eps. 0.0000e+00 
iter. 0006 | Fval. 1.2599e-01 | BICval. 1.2599e-01 | Gcrit_x. 3.5582e-03 | avg. eps. 0.0000e+00 
iter. 0007 | Fval. 1.2209e-01 | BICval. 1.2209e-01 | Gcrit_x. 1.0847e-03 | avg. eps. 0.0000e+00 
iter. 0008 | Fval. 1.2022e-01 | BICval. 1.2022e-01 | Gcrit_x. 9.4818e-04 | avg. eps. 0.0000e+00 
iter. 0009 | Fval. 1.1762e-01 | BICval. 1.1762e-01 | Gcrit_x. 8.7221e-04 | avg. eps. 0.0000e+00 
iter. 0010 | Fval. 1.1624e-01 | BICval. 1.1624e-01 | Gcrit_x. 3.5169e-04 | avg.


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.04791    0   99    0.08393    0.04791  42.9%     -   17s
H    0     0                       0.0838937    0.04791  42.9%     -   20s
H    0     0                       0.0838890    0.04791  42.9%     -   23s
     0     0    0.04791    0   99    0.08389    0.04791  42.9%     -   25s
     0     2    0.04791    0   99    0.08389    0.04791  42.9%     -   29s
     4     8    0.04955    2   97    0.08389    0.04791  42.9%  67.5   31s
    25    32    0.04986    6   95    0.08389    0.04889  41.7%   284   35s
    31    38    0.05079    7   93    0.08389    0.04889  41.7%   287   42s
    51    60    0.05335   10   87    0.08389    0.04889  41.7%   341   46s
    94   103    0.05747   17   77    0.08389    0.04889  41.7%   275   50s
   134   149    0.06104   23   65    0.08389    0.04889  41.7%   255   57s
   193   214    0.06209

     0     0    0.05705    0   44    0.07324    0.05705  22.1%     -   19s
     0     0    0.05734    0   44    0.07324    0.05734  21.7%     -   24s
     0     2    0.05734    0   44    0.07324    0.05734  21.7%     -   27s
    13    20    0.05806    4   41    0.07324    0.05734  21.7%   327   31s
    31    38    0.05862    6   39    0.07324    0.05734  21.7%   226   37s
H   33    38                       0.0732378    0.05734  21.7%   224   37s
    43    52    0.05916    7   40    0.07324    0.05734  21.7%   321   40s
   114   130    0.06152   12   34    0.07324    0.05734  21.7%   248   45s
   164   182    0.06334   17   28    0.07324    0.05734  21.7%   227   50s
   292   336    0.06890   31   14    0.07324    0.05734  21.7%   185   55s
H  392   402                       0.0732378    0.05734  21.7%   163   57s
*  412   403              42       0.0732356    0.05734  21.7%   164   58s
   458   478    0.05862    5   37    0.07324    0.05739  21.6%   161   60s
H  561   552             

    99    99    0.06962   11    1    0.06992    0.06715  3.96%   479   40s
   124   116    0.06770    5    7    0.06992    0.06740  3.60%   444   45s
   159   127    0.06900    9    3    0.06992    0.06740  3.60%   424   51s
   181   126    0.06962   11    1    0.06992    0.06740  3.60%   442   55s
   207   131    0.06962   11    1    0.06992    0.06740  3.60%   432   61s
   228   142    0.06803    6    6    0.06992    0.06740  3.60%   450   66s
   251   146    0.06868    8    4    0.06992    0.06740  3.60%   446   71s
   267   146    0.06962   11    1    0.06992    0.06740  3.60%   441   75s
   326   161    0.06962   11    1    0.06992    0.06741  3.59%   420   82s
   344   165    0.06803    6    6    0.06992    0.06741  3.59%   423   85s
   392   160    0.06962   11    1    0.06992    0.06741  3.59%   414   90s
   443   167    0.06803    6    6    0.06992    0.06755  3.38%   404   95s
   487   170    0.06933   10    2    0.06992    0.06755  3.38%   390  101s
   551   175    0.06962  

(CVXPY) Jul 24 04:10:49 PM: Solver (including time spent in interface) took 1.667e+01 seconds
 
no improvement after globalization technique... 
 
=> am (END)
 
new rep #2
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 4.0822e-13 | avg. eps. 0.0000e+00 
iter. 0002 | Fval. 1.6293e-01 | BICval. 1.6293e-01 | Gcrit_x. 3.1680e-03 | avg. eps. 0.0000e+00 
iter. 0003 | Fval. 1.5419e-01 | BICval. 1.5419e-01 | Gcrit_x. 4.7634e-03 | avg. eps. 0.0000e+00 
iter. 0004 | Fval. 1.4047e-01 | BICval. 1.4047e-01 | Gcrit_x. 5.4168e-03 | avg. eps. 0.0000e+00 
iter. 0005 | Fval. 1.3226e-01 | BICval. 1.3226e-01 | Gcrit_x. 3.7092e-03 | avg. eps. 0.0000e+00 
iter. 0006 | Fval. 1.2862e-01 | BICval. 1.2862e-01 | Gcrit_x. 1.2454e-03 | avg. eps. 0.0000e+00 
iter. 0007 | Fval. 1.2666e-01 | BICval. 1.2666e-01 | Gcrit_x. 1.0025e-03 | avg. eps. 0.0000e+00 
iter. 0008 | Fval. 1.2566e-01 | BICval. 1.2566e-01 | Gcrit_x. 3.4026e-04 | avg. eps. 0.0000e+00 
iter. 0009 | Fval. 1.2479e-01 |

...

  RHS range        [7e-03, 2e+02]

User MIP start produced solution with objective 0.0844246 (0.13s)
Loaded user MIP start with objective 0.0844246

Presolve removed 11984 rows and 3848 columns
Presolve time: 2.16s
Presolved: 8980 rows, 1491 columns, 241872 nonzeros
Variable types: 1386 continuous, 105 integer (105 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4038   -4.5064721e+02   2.233823e+05   0.000000e+00      5s
   10802   -1.0842870e+02   1.911954e+04   0.000000e+00     10s
   19917    4.7898967e-02   2.034314e+01   0.000000e+00     15s
   23246    5.4299558e-02   0.000000e+00   0.000000e+00     17s

Root relaxation: objective 5.429956e-02, 23246 iterations, 14.46 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.05430    0   80    0.08442    0.05430  35.7%     -   17s
H    0     0                    

   12614   -7.4984384e-01   1.752142e+04   0.000000e+00     30s
   16326    4.1554866e-02   1.268824e+03   0.000000e+00     35s
   20621    5.5649062e-02   8.282270e+01   0.000000e+00     40s
   23240    5.7480328e-02   0.000000e+00   0.000000e+00     44s

Root relaxation: objective 5.748033e-02, 23240 iterations, 38.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.05748    0   55    0.07671    0.05748  25.1%     -   44s
H    0     0                       0.0767062    0.05748  25.1%     -   49s
H    0     0                       0.0767060    0.05748  25.1%     -   53s
     0     0    0.05748    0   55    0.07671    0.05748  25.1%     -   54s
H    0     0                       0.0767060    0.05748  25.1%     -   58s
H    0     0                       0.0767055    0.05748  25.1%     -   62s
     0     2    0.05748    0   55    0.07671    0.05748  25.1%     -

H    0     0                       0.0750015    0.04624  38.4%     -   38s
H    0     0                       0.0750015    0.04624  38.4%     -   39s
     0     0    0.04624    0   79    0.07500    0.04624  38.4%     -   39s
     0     2    0.04624    0   79    0.07500    0.04624  38.4%     -   43s
     1     4    0.04624    1   79    0.07500    0.04624  38.4%  10.0   45s
    13    20    0.04711    4   79    0.07500    0.04711  37.2%   366   51s
    19    26    0.04723    5   77    0.07500    0.04714  37.2%   352   55s
H   31    38                       0.0750012    0.04714  37.2%   394   71s
H   33    38                       0.0750003    0.04714  37.1%   385   71s
    57    65    0.04963   10   71    0.07500    0.04714  37.1%   293   76s
    71    80    0.05103   12   67    0.07500    0.04714  37.1%   318   80s
H   88    96                       0.0749993    0.04714  37.1%   316   92s
   116   132    0.05540   19   53    0.07500    0.04714  37.1%   293   95s
H  174   194             

   606   481    0.07259   27    1    0.07289    0.06374  12.6%   139   76s
   770   608     cutoff   28         0.07289    0.06374  12.6%   131   81s
   892   650     cutoff   28         0.07289    0.06374  12.6%   133   85s
  1007   732    0.06446    5   23    0.07289    0.06446  11.6%   134   90s
  1191   891    0.06693   11   17    0.07289    0.06492  10.9%   131   96s

Explored 1293 nodes (186719 simplex iterations) in 120.07 seconds
Thread count was 12 (of 12 available processors)

Solution count 4: 0.0728902 0.0728902 0.0728959 0.0728959 

Time limit reached
Best objective 7.289024626592e-02, best bound 6.492165796763e-02, gap 10.9323%
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jul 24 05:32:22 PM: Problem status: user_limit
(CVXPY) Jul 24 05:32:22 PM: Optimal value: 7.289e-02
(C

(CVXPY) Jul 24 05:35:12 PM: Optimal value: 7.135e-02
(CVXPY) Jul 24 05:35:12 PM: Compilation took 3.092e+00 seconds
(CVXPY) Jul 24 05:35:12 PM: Solver (including time spent in interface) took 1.208e+02 seconds
 
 /!\ NEW BEST | F = 0.07135201936361593  /!\ 
 
OUTER #7
 
restart w/ F = 0.07135201936361593
 
local
 
iter. 0001 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0002 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0003 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0004 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0005 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0006 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter. 0007 | Fval. 7.0973e-02 | BICval. 7.0973e-02 | Gcrit_x. 0.0000e+00 | avg. eps. 0.0000e+00 
iter.

In [16]:
t_bb,fval_bb,repIDs_bb,locopt_bb,endval_bb,endval_single_run_bb = RRAM(mode='baratt-boyd')

=> baratt-boyd (START)
 
new rep #1
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 6.6742e-14 | avg. eps. 1.0000e+00 
iter. 0002 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 2.1927e-14 | avg. eps. 1.0000e+00 
iter. 0003 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 5.8609e-12 | avg. eps. 1.0000e+00 
iter. 0004 | Fval. 1.6856e-01 | BICval. 1.6879e-01 | Gcrit_x. 1.5995e-04 | avg. eps. 1.0000e+00 
iter. 0005 | Fval. 1.6567e-01 | BICval. 1.6681e-01 | Gcrit_x. 1.4580e-03 | avg. eps. 1.0000e+00 
iter. 0006 | Fval. 1.6350e-01 | BICval. 1.6544e-01 | Gcrit_x. 1.1509e-03 | avg. eps. 1.0000e+00 
iter. 0007 | Fval. 1.5648e-01 | BICval. 1.6179e-01 | Gcrit_x. 2.6176e-03 | avg. eps. 1.0000e+00 
iter. 0008 | Fval. 1.5404e-01 | BICval. 1.5858e-01 | Gcrit_x. 2.7659e-03 | avg. eps. 1.0000e+00 
iter. 0009 | Fval. 1.5186e-01 | BICval. 1.5570e-01 | Gcrit_x. 2.5091e-03 | avg. eps. 1.0000e+00 
iter. 0010 | Fval. 1.5076e-01 | BICval. 1.5368e-01 | Gcrit_x. 1.8330e-

iter. 0085 | Fval. 1.0426e-01 | BICval. 1.0498e-01 | Gcrit_x. 2.4655e-04 | avg. eps. 1.0000e+00 
iter. 0086 | Fval. 1.0305e-01 | BICval. 1.0447e-01 | Gcrit_x. 3.9554e-04 | avg. eps. 1.0000e+00 
iter. 0087 | Fval. 1.0248e-01 | BICval. 1.0391e-01 | Gcrit_x. 5.0065e-04 | avg. eps. 1.0000e+00 
iter. 0088 | Fval. 1.0175e-01 | BICval. 1.0324e-01 | Gcrit_x. 6.1958e-04 | avg. eps. 1.0000e+00 
iter. 0089 | Fval. 1.0096e-01 | BICval. 1.0247e-01 | Gcrit_x. 6.9406e-04 | avg. eps. 1.0000e+00 
iter. 0090 | Fval. 1.0037e-01 | BICval. 1.0175e-01 | Gcrit_x. 6.1528e-04 | avg. eps. 1.0000e+00 
iter. 0091 | Fval. 9.9928e-02 | BICval. 1.0131e-01 | Gcrit_x. 3.9892e-04 | avg. eps. 1.0000e+00 
iter. 0092 | Fval. 9.9391e-02 | BICval. 1.0086e-01 | Gcrit_x. 3.9246e-04 | avg. eps. 1.0000e+00 
iter. 0093 | Fval. 9.8559e-02 | BICval. 1.0028e-01 | Gcrit_x. 5.3445e-04 | avg. eps. 1.0000e+00 
iter. 0094 | Fval. 9.7227e-02 | BICval. 9.9390e-02 | Gcrit_x. 7.3068e-04 | avg. eps. 1.0000e+00 
iter. 0095 | Fval. 9.6390e-02 

(CVXPY) Jul 24 05:42:01 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 05:42:01 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 05:42:01 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 05:42:01 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 05:42:05 PM: Applying reduction GUROBI
(CVXPY) Jul 24 05:42:05 PM: Finished problem compilation (took 4.636e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 24 05:42:05 PM: Invoking solver GUROBI  to obtain a solution.
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical process

(CVXPY) Jul 24 05:45:12 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 05:45:12 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 05:45:12 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 05:45:15 PM: Applying reduction GUROBI
(CVXPY) Jul 24 05:45:15 PM: Finished problem compilation (took 2.978e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 24 05:45:15 PM: Invoking solver GUROBI  to obtain a solution.
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20910 rows, 5285 columns and 494215 nonzeros
Model finger

(CVXPY) Jul 24 05:48:20 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 05:48:20 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 05:48:20 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 05:48:21 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 05:48:23 PM: Applying reduction GUROBI
(CVXPY) Jul 24 05:48:23 PM: Finished problem compilation (took 2.789e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 24 05:48:23 PM: Invoking solver GUROBI  to obtain a solution.
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical process

...

-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 07:21:03 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Jul 24 07:21:03 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 07:21:03 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 07:21:03 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 07:21:03 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 07:21:05 PM: Applying reduction GUROBI
(CVXPY) Jul 24 07:21:05 PM: Finished problem compilation (took 2.396e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 24 07:2

(CVXPY) Jul 24 07:23:56 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 24 07:23:56 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 24 07:23:56 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 24 07:23:56 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 24 07:23:59 PM: Applying reduction GUROBI
(CVXPY) Jul 24 07:23:59 PM: Finished problem compilation (took 2.894e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 24 07:23:59 PM: Invoking solver GUROBI  to obtain a solution.
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical process

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20978 rows, 5353 columns and 496797 nonzeros
Model fingerprint: 0xafa4557a
Variable types: 5200 continuous, 153 integer (153 binary)
Coefficient statistics:
  Matrix range     [2e-06, 5e+00]
  Objective range  [2e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 3e+02]

User MIP start produced solution with objective 0.0824515 (0.17s)
Loaded user MIP start with objective 0.0824515

Presolve removed 12010 rows and 3872 columns
Presolve time: 2.22s
Presolved: 8968 rows, 1481 columns, 241731 nonzeros
Variable types: 1397 continuous, 84 integer (84 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    3602   -5.5129612e+02  

Variable types: 1393 continuous, 76 integer (76 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    6095   -4.1510958e+02   4.446796e+05   0.000000e+00      5s
   16038   -1.1507281e+01   6.656159e+03   0.000000e+00     10s
   26842    5.5640721e-02   0.000000e+00   0.000000e+00     15s

Root relaxation: objective 5.564072e-02, 26842 iterations, 13.12 seconds
Total elapsed time = 15.18s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.05564    0   73    0.08204    0.05564  32.2%     -   15s
H    0     0                       0.0820417    0.05564  32.2%     -   19s
H    0     0                       0.0820417    0.05564  32.2%     -   20s
     0     0    0.05569    0   73    0.08204    0.05569  32.1%     -   20s
     0     2    0.05569    0   73    0.08204    0.05569  32.1%     -   24s
     1     4    0.05569    1   73    

   21558    7.0404693e-02   0.000000e+00   0.000000e+00     12s

Root relaxation: objective 7.040469e-02, 21558 iterations, 9.84 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.07040    0   28    0.08144    0.07040  13.6%     -   11s
H    0     0                       0.0814368    0.07040  13.5%     -   13s
     0     0    0.07040    0   28    0.08144    0.07040  13.5%     -   13s
     0     2    0.07040    0   28    0.08144    0.07040  13.5%     -   17s
    13    20    0.07232    4   24    0.08144    0.07152  12.2%   477   21s
    31    38    0.07281    5   23    0.08144    0.07152  12.2%   873   27s
    37    44    0.07328    6   22    0.08144    0.07152  12.2%   814   31s
    83    98    0.07508   10   18    0.08144    0.07152  12.2%   453   35s
   156   175    0.07673   14   14    0.08144    0.07152  12.2%   358   41s
   250   264    0.07964   22    6   

In [17]:
t_maxmin,fval_maxmin,repIDs_maxmin,locopt_maxmin,endval_maxmin,endval_single_run_maxmin = RRAM(mode='maxmin')

=> maxmin (START)
 
new rep #1
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.6898e-01 | BICval. 1.6898e-01 | Gcrit_x. 3.2780e-13 | avg. eps. 2.1859e-04 
iter. 0002 | Fval. 1.6281e-01 | BICval. 1.6377e-01 | Gcrit_x. 1.9234e-03 | avg. eps. 1.1659e-01 
iter. 0003 | Fval. 1.5517e-01 | BICval. 1.5630e-01 | Gcrit_x. 5.1677e-03 | avg. eps. 3.6626e-01 
iter. 0004 | Fval. 1.5077e-01 | BICval. 1.5136e-01 | Gcrit_x. 2.4053e-03 | avg. eps. 1.5532e-01 
iter. 0005 | Fval. 1.4772e-01 | BICval. 1.4815e-01 | Gcrit_x. 2.1049e-03 | avg. eps. 4.0438e-01 
iter. 0006 | Fval. 1.3635e-01 | BICval. 1.3825e-01 | Gcrit_x. 4.6314e-03 | avg. eps. 1.0682e-01 
iter. 0007 | Fval. 1.3274e-01 | BICval. 1.3379e-01 | Gcrit_x. 2.1469e-03 | avg. eps. 9.2658e-02 
iter. 0008 | Fval. 1.2986e-01 | BICval. 1.3051e-01 | Gcrit_x. 1.6917e-03 | avg. eps. 6.4076e-02 
iter. 0009 | Fval. 1.2927e-01 | BICval. 1.2953e-01 | Gcrit_x. 5.6957e-04 | avg. eps. 5.8007e-02 
iter. 0010 | Fval. 1.2891e-01 | BICval. 1.2903e-01 | Gcrit_x. 2.9461e-04 | 

   20494   -5.1341899e+01   9.929816e+03   0.000000e+00     15s
   28922   -1.9347948e-02   4.289682e+03   0.000000e+00     20s
   30444    7.6436271e-03   0.000000e+00   0.000000e+00     22s

Root relaxation: objective 7.643627e-03, 30444 iterations, 19.45 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00764    0  234    0.08935    0.00764  91.4%     -   22s
     0     0    0.00764    0  234    0.08935    0.00764  91.4%     -   30s
     0     2    0.00764    0  234    0.08935    0.00764  91.4%     -   39s
     1     4    0.00764    1  234    0.08935    0.00764  91.4%  18.0   41s
    31    38    0.00959    7  229    0.08935    0.00801  91.0%   285   60s
    74    90    0.01100   14  224    0.08935    0.00801  91.0%   264   65s
   118   132    0.01292   22  218    0.08935    0.00801  91.0%   263   70s
   157   169    0.01549   31  206    0.08935    0.00801  

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20770 rows, 5145 columns and 488451 nonzeros
Model fingerprint: 0xb0b094d6
Variable types: 5133 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [3e-05, 5e+00]
  Objective range  [3e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-03, 3e+02]

User MIP start produced solution with objective 0.0980885 (0.12s)
Loaded user MIP start with objective 0.0980885

Presolve removed 11945 rows and 3809 columns
Presolve time: 1.55s
Presolved: 8825 rows, 1336 columns, 234000 nonzeros
Found heuristic solution: objective 0.0980885
Variable types: 1330 continuous, 6 integer (6 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual 

    94   101    0.06566   15   80    0.09513    0.05770  39.3%   228   66s
   150   174    0.06954   23   70    0.09513    0.05770  39.3%   204   70s
   273   305    0.07433   33   56    0.09513    0.05770  39.3%   149   75s
   391   409    0.07913   44   43    0.09513    0.05770  39.3%   134   80s
   517   549    0.08289   53   34    0.09513    0.05770  39.3%   142   85s
H  573   614                       0.0951336    0.05770  39.3%   143   87s
   639   677    0.08837   67   20    0.09513    0.05770  39.3%   142   90s
   719   744    0.09200   77   10    0.09513    0.05770  39.3%   139   98s
   808   844    0.06127   10   92    0.09513    0.05801  39.0%   135  101s
   960   996    0.06771   22   74    0.09513    0.05801  39.0%   134  106s
  1138  1183    0.07868   47   44    0.09513    0.05801  39.0%   131  111s
  1285  1323    0.08331   58   33    0.09513    0.05801  39.0%   128  115s
* 1308  1323             105       0.0951330    0.05801  39.0%   128  115s
* 1323  1323             

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jul 24 07:44:39 PM: Problem status: user_limit
(CVXPY) Jul 24 07:44:39 PM: Optimal value: 9.152e-02
(CVXPY) Jul 24 07:44:39 PM: Compilation took 1.699e+00 seconds
(CVXPY) Jul 24 07:44:39 PM: Solver (including time spent in interface) took 1.206e+02 seconds
 
 /!\ NEW BEST | F = 0.09152187666937206  /!\ 
 
OUTER #4
 
restart w/ F = 0.09152187666937206
 
local
 
... solver failed | early interruption 
 
 /!\ NEW BEST | F = 0.09152187666937206  /!\ 
 
global
 
-> computation of r_beta = 0.1 ; r_gamma = 0.1 ; p-norm with p = 1
 
ok for 10.0%
ok for 20.0%
ok for 30.0%
ok for 40.0%
ok for 50.0%
ok for 60.0%
ok for 70.0%
ok for 80.0%
ok for 90.0%
ok for 100.0%
Changed value of parameter BestObjStop to 0.08969143913598461
   Prev: 0.08971638623727758

...

Model fingerprint: 0x5a4d43fb
Variable types: 5340 continuous, 512 integer (512 binary)
Coefficient statistics:
  Matrix range     [2e-06, 5e+00]
  Objective range  [2e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 2e+02]

User MIP start produced solution with objective 0.0958651 (0.09s)
Loaded user MIP start with objective 0.0958651

Presolve removed 12105 rows and 3969 columns
Presolve time: 1.30s
Presolved: 9372 rows, 1883 columns, 261247 nonzeros
Variable types: 1541 continuous, 342 integer (342 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7584   -1.2311811e+03   1.909406e+05   0.000000e+00      5s
   20789   -4.5263759e-01   7.496422e+03   0.000000e+00     10s
   31019    3.9272127e-03   0.000000e+00   0.000000e+00     14s

Root relaxation: objective 3.927213e-03, 31019 iterations, 12.77 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

Coefficient statistics:
  Matrix range     [1e-05, 5e+00]
  Objective range  [5e-06, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 2e+02]

User MIP start produced solution with objective 0.0925382 (0.09s)
Loaded user MIP start with objective 0.0925382

Presolve removed 11969 rows and 3833 columns
Presolve time: 1.51s
Presolved: 8896 rows, 1407 columns, 237742 nonzeros
Variable types: 1359 continuous, 48 integer (48 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7998   -2.1608677e+02   1.930395e+06   0.000000e+00      5s
   19906    7.3542474e-02   0.000000e+00   0.000000e+00      9s

Root relaxation: objective 7.354247e-02, 19906 iterations, 7.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.07354    0   42    0.09254    0.07354  20.5%     -    9s
H    0     0                       0.

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20879 rows, 5254 columns and 492975 nonzeros
Model fingerprint: 0x1dfff9dc
Variable types: 5166 continuous, 88 integer (88 binary)
Coefficient statistics:
  Matrix range     [1e-05, 5e+00]
  Objective range  [8e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-03, 2e+02]

User MIP start produced solution with objective 0.0915845 (0.09s)
Loaded user MIP start with objective 0.0915845

Presolve removed 11973 rows and 3837 columns
Presolve time: 1.37s
Presolved: 8906 rows, 1417 columns, 238316 nonzeros
Variable types: 1363 continuous, 54 integer (54 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8130   -2.3381128e+02   9.975373e+05   0.000000e+00      5s
   22028    7.0372451e-02   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 7.037245e-02, 22028 ite

Presolve time: 1.38s
Presolved: 8897 rows, 1408 columns, 237804 nonzeros
Variable types: 1359 continuous, 49 integer (49 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7768   -2.4266721e+02   1.825951e+05   0.000000e+00      5s
   18613    7.2965356e-02   0.000000e+00   0.000000e+00      9s

Root relaxation: objective 7.296536e-02, 18613 iterations, 7.29 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.07297    0   41    0.09154    0.07297  20.3%     -    9s
H    0     0                       0.0915362    0.07297  20.3%     -   10s
     0     0    0.07297    0   41    0.09154    0.07297  20.3%     -   10s
     0     2    0.07297    0   41    0.09154    0.07297  20.3%     -   13s
    19    26    0.07433    4   41    0.09154    0.07433  18.8%   215   15s
    65    78    0.07797   10   31    0.09154    0.07500  


     0     0    0.07964    0   26    0.09151    0.07964  13.0%     -    9s
H    0     0                       0.0915056    0.07964  13.0%     -   10s
     0     0    0.08128    0   26    0.09151    0.08128  11.2%     -   10s
     0     2    0.08128    0   26    0.09151    0.08128  11.2%     -   12s
H   31    38                       0.0915056    0.08254  9.80%   1.0   14s
    79    92    0.08494    9   17    0.09151    0.08254  9.80%   5.7   15s
H  281   298                       0.0915056    0.08254  9.80%  61.0   19s
   297   317    0.09116   25    1    0.09151    0.08254  9.80%  61.7   20s
   429   394    0.09081   24    2    0.09151    0.08306  9.23%   113   25s
   537   448    0.09116   25    1    0.09151    0.08306  9.23%   125   30s
   633   465    0.09116   25    1    0.09151    0.08306  9.23%   136   35s
   817   581    0.09116   25    1    0.09151    0.08306  9.23%   127   40s
   939   605    0.09081   24    2    0.09151    0.08306  9.23%   136   45s
  1023   650    0.09116 

In [26]:
np.mean((np.array(endval_single_run_sm)-np.array(endval_sm))/np.abs(np.array(endval_single_run_sm)))

0.047949813417910506

In [19]:
locopt_sm

[True, True, True, True, True]

In [20]:
np.mean(endval_sm)

0.08134131893476958

In [21]:
np.mean(endval_maxmin)

0.08534406901779606

In [23]:
np.mean(endval_am)

0.07958722067880594

In [24]:
np.mean(endval_bb)

0.08566852740314404